In [ ]:
"""Exploring and preprocessing energy demand."""

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from loguru import logger

from exploratory import df_energy

In [ ]:
# Removing nan or zero columns and forecast columns
columns_to_check = ['generation fossil coal-derived gas', 'generation fossil oil shale', 
                     'generation fossil peat', 'generation geothermal', 
                     'generation hydro pumped storage aggregated', 'generation marine', 
                     'generation wind offshore', 'forecast wind offshore eday ahead',
                     'total load forecast', 'forecast solar day ahead',
                     'forecast wind onshore day ahead']

# Initialize lists to store the percentages
nan_percentages = []
zero_percentages = []

for column in columns_to_check:
    # Count NaN values
    nan_count = df_energy[column].isna().sum()
    # Count zeros
    zero_count = (df_energy[column] == 0).sum()
    # Total number of values in the column
    total_values = len(df_energy[column])
    
    # Calculate the percentage of zeros and NaNs
    nan_percentage = (nan_count / total_values) * 100
    zero_percentage = (zero_count / total_values) * 100
    
    nan_percentages.append(nan_percentage)
    zero_percentages.append(zero_percentage)

# Create a bar plot
plt.bar(columns_to_check, nan_percentages, label='NaN Percentage')
plt.bar(columns_to_check, zero_percentages, bottom=nan_percentages, label='Zero Percentage')

# Add labels and title
plt.xlabel('Columns')
# Rotate x-axis labels to be vertical
plt.xticks(rotation='vertical')
plt.ylabel('Percentage')
plt.title('Percentage of NaNs and Zeros in Each Column')
plt.legend()

# Show the plot
plt.show()


In [ ]:
# We can see either all values are zero/nan so drop
df_energy = df_energy.drop(['generation fossil coal-derived gas','generation fossil oil shale', 
                            'generation fossil peat', 'generation geothermal', 
                            'generation hydro pumped storage aggregated', 'generation marine', 
                            'generation wind offshore', 'forecast wind offshore eday ahead',
                            'total load forecast', 'forecast solar day ahead',
                            'forecast wind onshore day ahead'], 
                            axis=1)

df_energy.describe().round(2)

In [ ]:
# ensure time axis is parsed correctly
df_energy['time'] = pd.to_datetime(df_energy['time'], utc=True, infer_datetime_format=True)
df_energy = df_energy.set_index('time')

df_energy.info()

In [ ]:
# Find NaNs and duplicates in df_energy
logger.info(f'There are {df_energy.isnull().values.sum()} missing values or NaNs in df_energy.')

temp_energy = df_energy.duplicated(keep='first').sum()

logger.info(f'There are {temp_energy} duplicate rows in df_energy based on all columns.')

# determine which rows to are Nans since timeseries shouldn't drop

df_energy.isnull().sum(axis=0)

In [ ]:
# Define a function to plot different types of time-series

def plot_column_and_user_series(
    df: pd.DataFrame = None,
    column: str = None,
    series: pd.Series = pd.Series([]),
    label: str = None,
    ylabel: str = None,
    title: str = None,
    start: int = 0,
    end: int = None,
) -> plt.Axes:
    """Plots a time-series either from a specified column in a DataFrame or from a custom pandas Series.

    Parameters:
    - df (pd.DataFrame, optional): The DataFrame containing the time-series data. Defaults to None.
    - column (str, optional): The name of the column in the DataFrame to plot. Defaults to None.
    - series (pd.Series, optional): A custom pandas Series to plot. Defaults to an empty Series.
    - label (str, optional): The label for the plotted series. Defaults to None.
    - ylabel (str, optional): The label for the y-axis. Defaults to None.
    - title (str, optional): The title of the plot. Defaults to None.
    - start (int, optional): The start index for slicing the DataFrame or Series. Defaults to 0.
    - end (int, optional): The end index for slicing the DataFrame or Series. Defaults to None.

    Returns:
    - plt.Axes: The matplotlib Axes object of the plot.
    """
    sns.set_theme()
    fig, ax = plt.subplots(figsize=(30, 12))
    ax.set_xlabel("Time", fontsize=16)
    if column:
        ax.plot(df[column][start:end], label=label)
        ax.set_ylabel(ylabel, fontsize=16)
    if series.any():
        ax.plot(series, label=label)
        ax.set_ylabel(ylabel, fontsize=16)
    if label:
        ax.legend(fontsize=16)
    if title:
        ax.set_title(title, fontsize=24)
    ax.grid(True)
    return ax



In [ ]:
# Zoom into the plot of the hourly (actual) total load

ax = plot_column_and_user_series(df=df_energy, column='total load actual', ylabel='Total Load (MWh)',
                 title='Actual Total Load (First 2 weeks - Original)', end=24*7*2)
plt.show()

In [ ]:
# Since the nan gap is small and the timeseries pattern is consistent, interpolation is appropriate
# check if the other null values correspond with this time
df_energy[df_energy.isnull().any(axis=1)]
# The null values in the columns describing the type of energy generation mostly coincide with each other.
# The null values in 'actual total load' appear in other rows as well. 


In [ ]:
# In order to handle the null values in df_energy, use a linear interpolation with a forward direction. 
# Possibly other kinds of interpolation would be better; nevertheless, we prefer to use the simplest model possible. 
# Only a small part of input data will be noisy and it will not affect performance noticeably.
df_energy.interpolate(method='linear', limit_direction='forward', inplace=True, axis=0)

In [ ]:
# It look like df_energy has been cleaned successfully and is ready for further use as input into our model. 
# The 1-4 zeroes in the columns which have to do with energy generation by type should not concern us very much. 
# The 'generation hydro pumped storage consumption' may look suspicious, but we should have in mind 
# that this type of energy is only used for load balancing, being consumed when in peak energy demands.
logger.info('Non-zero values in each column:\n', df_energy.astype(bool).sum(axis=0), sep='\n')